#### Importing libraries

In [2]:
# imporing libraries
import pandas as pd
import numpy as np
import os
import ydata_profiling as yd

In [3]:
# importing data
path = r'C:\Users\Richárd\Desktop\CareerFoundry\Data Immersion\Achievement 6\project\Data'
raw_data = pd.read_csv(os.path.join(path, 'supply_chain_raw.csv'), encoding = 'latin-1')

In [5]:
raw_data['Order Profit Per Order']

count    180519.000000
mean         21.974989
std         104.433526
min       -4274.979980
25%           7.000000
50%          31.520000
75%          64.800003
max         911.799988
Name: Order Profit Per Order, dtype: float64

In [34]:
raw_data.columns

Index(['Type', 'Days for shipping (real)', 'Days for shipment (scheduled)',
       'Benefit per order', 'Sales per customer', 'Delivery Status',
       'Late_delivery_risk', 'Category Id', 'Category Name', 'Customer City',
       'Customer Country', 'Customer Email', 'Customer Fname', 'Customer Id',
       'Customer Lname', 'Customer Password', 'Customer Segment',
       'Customer State', 'Customer Street', 'Customer Zipcode',
       'Department Id', 'Department Name', 'Latitude', 'Longitude', 'Market',
       'Order City', 'Order Country', 'Order Customer Id',
       'order date (DateOrders)', 'Order Id', 'Order Item Cardprod Id',
       'Order Item Discount', 'Order Item Discount Rate', 'Order Item Id',
       'Order Item Product Price', 'Order Item Profit Ratio',
       'Order Item Quantity', 'Sales', 'Order Item Total',
       'Order Profit Per Order', 'Order Region', 'Order State', 'Order Status',
       'Order Zipcode', 'Product Card Id', 'Product Category Id',
       'Product De

#### Cleaning and consistency checks

In [35]:
# omitting unecessary columns
# omitted due being irrelevant: ['Customer Email', 'Customer Fname', 'Customer Lname', 'Customer Password', 'Customer Zipcode', 'Latitude', 'Longitude', 'Order Item Cardprod Id', 'Order Zipcode', 'Product Image', 'Product Description']
# omitted duplicate columns : ['Order Customer Id', 'Product Category Id', 'Order Item Product Price', 'Order Profit Per Order', 'Order Item Total',]
supply_chain = raw_data[['Type', 'Days for shipping (real)', 'Days for shipment (scheduled)',
       'Benefit per order', 'Sales per customer', 'Delivery Status',
       'Late_delivery_risk', 'Category Id', 'Category Name', 'Customer City',
       'Customer Country', 'Customer Id', 'Customer Segment',
       'Customer State', 'Department Id', 'Department Name', 'Market',
       'Order City', 'Order Country', 
       'order date (DateOrders)', 'Order Id', 'Order Item Discount', 'Order Item Discount Rate', 'Order Item Id',
       'Order Item Profit Ratio',
       'Order Item Quantity', 'Sales',
       'Order Region', 'Order State', 'Order Status', 'Product Card Id', 'Product Name', 'Product Price',
       'Product Status', 'shipping date (DateOrders)', 'Shipping Mode']]

In [36]:
# checking for duplicates
# no duplicates found
supply_chain.duplicated().value_counts()

False    180519
dtype: int64

In [37]:
# checking for missing values
# no missing values found
supply_chain.isnull().value_counts()

Type   Days for shipping (real)  Days for shipment (scheduled)  Benefit per order  Sales per customer  Delivery Status  Late_delivery_risk  Category Id  Category Name  Customer City  Customer Country  Customer Id  Customer Segment  Customer State  Department Id  Department Name  Market  Order City  Order Country  order date (DateOrders)  Order Id  Order Item Discount  Order Item Discount Rate  Order Item Id  Order Item Profit Ratio  Order Item Quantity  Sales  Order Region  Order State  Order Status  Product Card Id  Product Name  Product Price  Product Status  shipping date (DateOrders)  Shipping Mode
False  False                     False                          False              False               False            False               False        False          False          False             False        False             False           False          False            False   False       False          False                    False     False                False              

In [38]:
# negative profit values
# these negative values are suspicious, and there is no clear indication what they mean
# However they make up 19% of total entries so removing or modifying them would be wrong, this is a situation where data owner input would be very valuable
neg_profit_df = supply_chain[supply_chain['Benefit per order'] <= 0]
neg_profit_df['Sales'].corr(neg_profit_df['Product Price'])

0.7830126027871401

In [39]:
neg_profit_df[['Order Item Profit Ratio', 'Benefit per order']].describe()

,Order Item Profit Ratio,Benefit per order
count,34961.000000,34961.000000
mean,-0.603989,-111.082273
std,0.626538,158.916698
min,-2.750000,-4274.979980
25%,-0.780000,-144.250000
50%,-0.330000,-53.290001
75%,-0.150000,-17.990000
max,0.000000,0.000000


In [40]:
supply_chain[['Order Item Profit Ratio', 'Benefit per order']].describe()

,Order Item Profit Ratio,Benefit per order
count,180519.000000,180519.000000
mean,0.120647,21.974989
std,0.466796,104.433526
min,-2.750000,-4274.979980
25%,0.080000,7.000000
50%,0.270000,31.520000
75%,0.360000,64.800003
max,0.500000,911.799988


In [41]:
#### Renaming columns
supply_chain = supply_chain.rename(columns = {'Type' : 'type', 'Days for shipping (real)' : 'shipping_days_real', 'Days for shipment (scheduled)' : 'shipping_days_scheduled',
       'Benefit per order' : 'profit_per_order', 'Sales per customer' : 'sales_per_customer', 'Delivery Status' : 'delivery_status',
       'Late_delivery_risk' : 'late_delivery_risk', 'Category Id' : 'category_id', 'Category Name' : 'category_name', 'Customer City' : 'customer_city',
       'Customer Country' : 'customer_country', 'Customer Id' : 'customer_id', 'Customer Segment' : 'customer_segment',
       'Customer State' : 'customer_state', 'Department Id' : 'department_id', 'Department Name' : 'department_name', 'Market' : 'market',
       'Order City' : 'order_city', 'Order Country' : 'order_country', 
       'order date (DateOrders)' : 'order_date', 'Order Id' : 'order_id', 'Order Item Discount' : 'order_item_discount', 'Order Item Discount Rate' : 'order_item_discount_rate', 'Order Item Id': 'order_item_id',
       'Order Item Profit Ratio' : 'order_item_profit_ratio',
       'Order Item Quantity' : 'order_item_quantity', 'Sales' : 'sales',
       'Order Region' : 'order_region', 'Order State' : 'order_state', 'Order Status' : 'order_status', 'Product Card Id' : 'product_id_card', 'Product Name' : 'product_name', 'Product Price' : 'product_price',
       'Product Status' : 'product_status', 'shipping date (DateOrders)' : 'shipping_date', 'Shipping Mode' : 'shipping_mode'})

In [42]:
# creating data profile
clean_profile = yd.ProfileReport(supply_chain)
clean_profile.to_file("clean_supply_chain_profile.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [43]:
# exporting df to pickle
supply_chain.to_pickle(os.path.join(path, 'clean_supply_chain.pkl'))